In [1]:
# ssh variables

import os
from sshtunnel import SSHTunnelForwarder
import pg as db
import pandas as pd
import re
from sqlalchemy import create_engine
import numpy as np
import plotly as plotly
import plotly.express as px
%matplotlib inline

host = 'prd-db.spot-internal'
localhost = '127.0.0.1'
ssh_username = os.environ['SPOT_SSH_USER']
ssh_private_key = '/Users/david/.ssh/id_rsa'
ssh_private_key_pwd = os.environ['SSH_PWD']

intermediary=os.environ['SPOT_SSH_SERVER']
user=os.environ['SPOT_DB_USER']
password=os.environ['SPOT_DB_PW']

database='trace_wood_delawareconnector'

def query(q):
    with SSHTunnelForwarder(
    (intermediary, 22), 
    ssh_username=ssh_username,
    ssh_pkey=ssh_private_key, 
    ssh_private_key_password=ssh_private_key_pwd,
    remote_bind_address=(host,5432)) as server:

        connstring = 'postgresql://{user}:{pwd}@{host}:{localport}/{database}'.format(
            user=user,
            pwd=password,
            host=localhost,
            localport=server.local_bind_port,
            database=database)
        
        engine = create_engine(connstring)

        try:
            result = pd.read_sql_query(q, engine)
        finally:
            engine.dispose()

        return result;

In [4]:
db_dynamic_types = query('select * from dynamic_type_definition')

In [5]:
db_dynamic_types.head()

,id,table_name,description,slug,created_at,updated_at,deleted_at,category,subcategory,show_in_menu,...,template_name,workflow_enabled,workflow_id,previous_slug,html_template_uri,version,public_version,viewer_group_id,creator_group_id,default_assignee_id
0,a6a5e027-ba2d-4774-aa68-66eb4b2d3577,_inspectionreport_description_two_subform,2.0 Action Items,inspection-report-description-two-subform,2019-03-21 13:40:15.406000+00:00,2019-06-07 14:23:54.865000+00:00,NaT,Forms,Wood Subform,False,...,None,False,None,None,None,74,1,88888888-8888-8888-8888-888888888888,88888888-8888-8888-8888-888888888888,00000000-0000-0000-0000-000000000001
1,248b9d96-3e27-4ba0-8da4-29cc4ad3e24a,_daily_inspection_report_and_summary_of_pipe_w...,PL0732: Daily Report and Summary of Pipe Welds...,daily-inspection-report-and-summary-of-pipe-welds,2019-03-21 12:58:58.045000+00:00,2019-08-12 18:02:37.114000+00:00,NaT,Forms,Exxon PL,True,...,PL0732 FINAL..pdf,True,88ac6388-5e5f-4a06-8df7-5d3f220e34b2,None,None,713,1,88888888-8888-8888-8888-888888888888,88888888-8888-8888-8888-888888888888,00000000-0000-0000-0000-000000000001
2,9f83f4e3-fe2c-4203-9d57-e55adbcf1740,_jeeping_daily_report,C12: Holiday Detection Report,jeeping-daily-report,2019-03-18 20:34:39.458000+00:00,2019-08-13 14:20:04.607000+00:00,NaT,Forms,Wood Pipeline,True,...,C12- Holiday Detection Report (1) (2).pdf,True,9334b10f-b906-493e-896f-f69b89da5ad7,None,None,541,1,88888888-8888-8888-8888-888888888888,88888888-8888-8888-8888-888888888888,00000000-0000-0000-0000-000000000001
3,c91fc4bc-5c31-4dc8-978a-8c1225b9ca88,_subform_welder_qualification_log,Subform Welder Qualification Log,subform-welder-qualification-log,2019-03-18 20:18:11.879000+00:00,2019-07-11 16:06:19.564000+00:00,NaT,Forms,Exxon Subform,False,...,None,False,None,None,None,430,1,88888888-8888-8888-8888-888888888888,88888888-8888-8888-8888-888888888888,00000000-0000-0000-0000-000000000001
4,29f42027-0ae7-415f-874c-1e52a65fcdea,_hvac0102_hvac_packaged_system_installation,HVAC-01-02: HVAC Packaged System Installation,hvac-01-02-hvac-packaged-system-installation,2019-04-30 03:31:04.235000+00:00,2019-06-12 12:32:03.573000+00:00,NaT,Forms,Review ITR Submissions,True,...,HVAC-01-02.pdf,True,9334b10f-b906-493e-896f-f69b89da5ad7,None,None,492,1,88888888-8888-8888-8888-888888888888,88888888-8888-8888-8888-888888888888,00000000-0000-0000-0000-000000000001


In [7]:
db_dynamic_types[db_dynamic_types['description'].apply(lambda x:('Chief' in x))]

,id,table_name,description,slug,created_at,updated_at,deleted_at,category,subcategory,show_in_menu,...,template_name,workflow_enabled,workflow_id,previous_slug,html_template_uri,version,public_version,viewer_group_id,creator_group_id,default_assignee_id
128,21e5133a-bddb-4e62-99d8-38e22801c9c9,_chief_daily_report,Chief Daily Report,chief-daily-report,2018-12-14 18:56:14.216000+00:00,2019-08-12 18:02:35.682000+00:00,NaT,Forms,Wood Facilities,True,...,Chief Daily Report (1).pdf,True,5d4bfdfc-0b8b-41d1-8bdb-fce72df0cb8c,None,1y2FcnJR8xaDVmt0VAdRmGh86eEmTVZo9,688,1,88888888-8888-8888-8888-888888888888,88888888-8888-8888-8888-888888888888,00000000-0000-0000-0000-000000000001


In [8]:
db_chief_daily_report = query('select * from _chief_daily_report')

In [9]:
db_chief_daily_report.head()

,id,type,created_at,updated_at,deleted_at,description,friendly_number,folder_id,modified_by,location_id,...,company_rep_signature_date,locked,status_id,region,pipeline_no,state,location_corridor,cost_center,project_manager,workflow_history
0,5b5db719-4147-49c9-8686-65ba524c01f8,_chief_daily_report,2019-04-27 18:03:26.424000+00:00,2019-08-15 20:32:05.156000+00:00,NaT,Chief Daily Report 7,7,1opM1eZ1Tb9SLbOaRcUYiZTodAUWW6UKX,92012b84-d657-46c9-8d16-e5efb9dbc718,None,...,None,2019-04-27 19:48:32.095000+00:00,fcf8c015-515d-4f9f-b591-2a2e49b21d5b,"West, TX",Wink ST,TX,DC-Fac,130754,Brian Kuta,"[{'date': '2019-04-27T17:58:06.800Z', 'statusI..."
1,0ef9475e-affc-427d-b4b4-f7e8e885bce2,_chief_daily_report,2019-03-09 21:51:46.677000+00:00,2019-08-15 20:31:56.701000+00:00,NaT,Chief Daily Report 2,2,19YOGRFCTjuZl9znHJ_-YYf5dBMUJ4EEd,92012b84-d657-46c9-8d16-e5efb9dbc718,None,...,None,2019-03-09 21:52:11.400000+00:00,fcf8c015-515d-4f9f-b591-2a2e49b21d5b,None,None,None,None,None,None,"[{'date': '2019-03-09T21:52:52.900Z', 'statusI..."
2,c67e00a5-3f2b-42cf-b9a8-9c693ed18804,_chief_daily_report,2019-04-27 18:03:27.804000+00:00,2019-08-15 20:32:21.225000+00:00,NaT,Chief Daily Report 8,8,1Ju2g6fNhV8NzZk-PZxOWN0GgiHfpoT5N,92012b84-d657-46c9-8d16-e5efb9dbc718,None,...,None,2019-04-27 19:36:48.868000+00:00,fcf8c015-515d-4f9f-b591-2a2e49b21d5b,"West, TX",Wink ST,TX,DC-Fac,130754,Brian Kuta,"[{'date': '2019-04-27T17:59:39.000Z', 'statusI..."
3,82f1bf38-a10e-41a0-bbe4-168c47fc5a10,_chief_daily_report,2019-01-28 20:15:37.515000+00:00,2019-01-29 20:58:10.890000+00:00,2019-01-29 20:58:10.910000+00:00,Chief Daily Report 1,1,None,d9d06a2d-7044-4147-a8f2-cf6d3eb5cb86,None,...,None,NaT,be8f911e-d496-4ed7-94b8-49bd5fa16e93,None,None,None,None,None,None,None
4,aabb5b6e-eb4b-4edc-889a-27dd525fbbf3,_chief_daily_report,2019-01-24 22:48:09.902000+00:00,2019-01-29 20:58:29.124000+00:00,2019-01-29 20:58:29.141000+00:00,Chief Daily Report 1,1,None,bd169b15-6bbc-4514-9cb0-d919d44f353c,None,...,None,NaT,be8f911e-d496-4ed7-94b8-49bd5fa16e93,None,None,None,None,None,None,None


In [10]:
db_statuses = query('select * From dynamic_workflow_status')

In [11]:
db_statuses.head()

,id,name,created_at,updated_at,deleted_at
0,be8f911e-d496-4ed7-94b8-49bd5fa16e93,Pending,2019-02-07 16:30:29.189000+00:00,2019-02-07 16:30:29.189000+00:00,None
1,e7b7c5cf-a43d-4940-bf1d-1e0da7531a8e,Submitted,2019-02-07 16:30:29.198000+00:00,2019-02-07 16:30:29.198000+00:00,None
2,0c0854e7-85a8-4c26-b034-808f3bc2db0b,Re-Submitted,2019-02-07 16:30:29.216000+00:00,2019-02-07 16:30:29.216000+00:00,None
3,996356d1-1d09-477a-98fc-d3189b5f4a7e,Wood Rejected,2019-02-07 16:30:29.208000+00:00,2019-03-12 15:56:45.083000+00:00,None
4,5e299e51-e408-46a4-9df0-37cdbdc478e4,Wood Approved,2019-02-07 16:30:29.223000+00:00,2019-03-12 15:56:53.446000+00:00,None


In [25]:
plotvalues = pd.DataFrame(db_chief_daily_report.merge(db_statuses[['id','name']].rename(columns={'id':'status_id','name':'status_name'}), on='status_id')['status_name'].value_counts()).reset_index()

In [27]:
plotvalues.head()

,index,status_name
0,Sr. Approved,122
1,Pending,5
2,Wood Rejected,1


In [37]:
px.bar(data_frame=plotvalues.rename_axis({'index':'Status','status_name':'Count'},axis=1),x='Status',y='Count',title='Daily Chief Report Current Status')

In [36]:
plotvalues.rename_axis({'index':'Status','status_name':'Count'},axis=1)

,Status,Count
0,Sr. Approved,122
1,Pending,5
2,Wood Rejected,1
